In [1]:
import geopandas as gpd
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-dark")

In [2]:
df = gpd.read_file(r'data/irrigation/Div4_Irrig_2015.shp')
df.drop(['SW_WDID1', 'SW_WDID2',
       'SW_WDID3', 'SW_WDID4', 'SW_WDID5', 'SW_WDID6', 'SW_WDID7', 'SW_WDID8',
       'SW_WDID9', 'GW_TYPE1', 'GW_ID1', 'GW_CLASS1', 'GW_TYPE2', 'GW_ID2',
       'GW_CLASS2', 'GW_TYPE3', 'GW_ID3', 'GW_CLASS3', 'GW_TYPE4', 'GW_ID4',
       'GW_CLASS4', 'GW_TYPE5', 'GW_ID5', 'GW_CLASS5', 'GW_TYPE6', 'GW_ID6',
       'GW_CLASS6', 'GW_TYPE7', 'GW_ID7', 'GW_CLASS7', 'GW_TYPE8', 'GW_ID8',
       'GW_CLASS8', 'GW_TYPE9', 'GW_ID9', 'GW_CLASS9', 'GW_TYPE10', 'GW_ID10',
       'GW_CLASS10', 'GW_TYPE11', 'GW_ID11', 'GW_CLASS11', 'GW_TYPE12',
       'GW_ID12', 'GW_CLASS12', 'GW_TYPE13', 'GW_ID13', 'GW_CLASS13',
       'GW_TYPE14', 'GW_ID14', 'GW_CLASS14', 'GW_TYPE15', 'GW_ID15',
       'GW_CLASS15', 'GW_TYPE16', 'GW_ID16', 'GW_CLASS16', 'GW_TYPE17',
       'GW_ID17', 'GW_CLASS17', 'GW_TYPE18', 'GW_ID18', 'GW_CLASS18',
       'GW_TYPE19', 'GW_ID19', 'GW_CLASS19', 'GW_TYPE20', 'GW_ID20',
       'GW_CLASS20', 'IRR', 'COMMENTS', 'FARM_UNIT'], axis=1, inplace=True)
qual_data =['CROP_TYPE', 'CROP_SRC', 'PERENNIAL', 'IRRIG_TYPE']
feat_to_predict = 'CROP_TYPE'

In [3]:
ctype = LabelEncoder()
csource = LabelEncoder()
perennial = LabelEncoder()
itype = LabelEncoder()
d2 = df.copy()
d2[qual_data[0]] = ctype.fit_transform(df[qual_data[0]].values)
d2[qual_data[1]] = csource.fit_transform(df[qual_data[1]].values)
d2[qual_data[2]] = perennial.fit_transform(df[qual_data[2]].values)
d2[qual_data[3]] = itype.fit_transform(df[qual_data[3]].values)

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [5]:
X = d2.drop(['CROP_TYPE', 'geometry'], axis=1)
y = d2['CROP_TYPE'] #need to LE earlier on with all the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=86)

In [6]:
pipe = Pipeline([('scaler', MinMaxScaler()), ('classifier', SVC())])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)

0.7542877995377691

In [7]:
pipe.score(X_test,y_test)

0.7529182879377432

In [8]:
y_hat = ctype.inverse_transform(pipe.predict(X_test))
labels = set(y_hat)

In [9]:
df_train = pd.merge(X_train, df, left_index=True, right_index=True)
df_test = pd.merge(X_test, df, left_index=True, right_index=True)
df_test['y_hat'] = y_hat
df_test['correct'] = 'Incorrect'
df_test.loc[df_test[df_test.CROP_TYPE == df_test.y_hat].index,'correct'] = 'Correct'

https://cdss.colorado.gov/gis-data/division-4-gunnison

In [10]:
df_test.to_file('data/test_preds.shp')
df_train.to_file('data/train_data.shp')

C:\Users\FAC_PI~1\AppData\Local\Temp/ipykernel_3728/1168737546.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df_test.to_file('data/test_preds.shp')
C:\Users\FAC_PI~1\AppData\Local\Temp/ipykernel_3728/1168737546.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df_train.to_file('data/train_data.shp')
